In [25]:
https://www.pyscoop.com/building-a-simple-blockchain-in-python/

https://www.pyscoop.com/building-a-simple-blockchain-in-python-part-2/

SyntaxError: invalid syntax (<ipython-input-25-0e8cabcea212>, line 1)

In [17]:
## import librairies for client blockchains

In [27]:
import time
import hashlib


class Block(object):

    def __init__(self, index, proof, previous_hash, transactions, timestamp=None):
        self.index = index
        self.proof = proof
        self.previous_hash = previous_hash
        self.transactions = transactions
        self.timestamp = timestamp or time.time()

    @property
    def get_block_hash(self):
        block_string = "{}{}{}{}{}".format(self.index, self.proof, self.previous_hash, self.transactions, self.timestamp)
        return hashlib.sha256(block_string.encode()).hexdigest()

    def __repr__(self):
        return "{} - {} - {} - {} - {}".format(self.index, self.proof, self.previous_hash, self.transactions, self.timestamp)


class BlockChain(object):

    def __init__(self):
        self.chain = []
        self.current_node_transactions = []
        self.nodes = set()
        self.create_genesis_block()

    @property
    def get_serialized_chain(self):
        return [vars(block) for block in self.chain]

    def create_genesis_block(self):
        self.create_new_block(proof=0, previous_hash=0)

    def create_new_block(self, proof, previous_hash):
        block = Block(
            index=len(self.chain),
            proof=proof,
            previous_hash=previous_hash,
            transactions=self.current_node_transactions
        )
        self.current_node_transactions = []  # Reset the transaction list

        self.chain.append(block)
        return block

    @staticmethod
    def is_valid_block(block, previous_block):
        if previous_block.index + 1 != block.index:
            return False

        elif previous_block.get_block_hash != block.previous_hash:
            return False

        elif not BlockChain.is_valid_proof(block.proof, previous_block.proof):
            return False

        elif block.timestamp <= previous_block.timestamp:
            return False

        return True

    def create_new_transaction(self, sender, recipient, amount):
        self.current_node_transactions.append({
            'sender': sender,
            'recipient': recipient,
            'amount': amount
        })
        return True

    @staticmethod
    def is_valid_transaction():
        # Not Implemented
        pass

    @staticmethod
    def create_proof_of_work(previous_proof):
        """
        Generate "Proof Of Work"
        A very simple `Proof of Work` Algorithm -
            - Find a number such that, sum of the number and previous POW number is divisible by 7
        """
        proof = previous_proof + 1
        while not BlockChain.is_valid_proof(proof, previous_proof):
            proof += 1

        return proof

    @staticmethod
    def is_valid_proof(proof, previous_proof):
        return (proof + previous_proof) % 7 == 0

    @property
    def get_last_block(self):
        return self.chain[-1]

    def is_valid_chain(self):
        """
        Check if given blockchain is valid
        """
        previous_block = self.chain[0]
        current_index = 1

        while current_index < len(self.chain):

            block = self.chain[current_index]

            if not self.is_valid_block(block, previous_block):
                return False

            previous_block = block
            current_index += 1

        return True

    def mine_block(self, miner_address):
        # Sender "0" means that this node has mined a new block
        # For mining the Block(or finding the proof), we must be awarded with some amount(in our case this is 1)
        self.create_new_transaction(
            sender="0",
            recipient=miner_address,
            amount=1,
        )

        last_block = self.get_last_block

        last_proof = last_block.proof
        proof = self.create_proof_of_work(last_proof)

        last_hash = last_block.get_block_hash
        block = self.create_new_block(proof, last_hash)

        return vars(block)  # Return a native Dict type object

    def create_node(self, address):
        self.nodes.add(address)
        return True

    @staticmethod
    def get_block_object_from_block_data(block_data):
        return Block(
            block_data['index'],
            block_data['proof'],
            block_data['previous_hash'],
            block_data['transactions'],
            timestamp=block_data['timestamp']
        )

In [28]:
blockchain = BlockChain()  
  
print(">>>>> Before Mining...")  
print(blockchain.chain)  
  
last_block = blockchain.get_last_block  
last_proof = last_block.proof  
proof = blockchain.create_proof_of_work(last_proof)  

# Sender "0" means that this node has mined a new block  
# For mining the Block(or finding the proof), we must be awarded with some amount(in our case this is 1)  

blockchain.create_new_transaction(  
    sender="0",  
    recipient="address_x",  
    amount=1,  
)  

last_hash = last_block.get_block_hash  
block = blockchain.create_new_block(proof, last_hash)  
  
print(">>>>> After Mining...")  
print(blockchain.chain)

>>>>> Before Mining...
[0 - 0 - 0 - [] - 1603729935.6986682]
>>>>> After Mining...
[0 - 0 - 0 - [] - 1603729935.6986682, 1 - 7 - 820f6244498f9b501c6cf572da5f497a40c7f6662abf502bf646008cced21642 - [{'sender': '0', 'recipient': 'address_x', 'amount': 1}] - 1603729935.700188]


In [29]:
import time  
import hashlib  
  
class Block(object):  
    ...  
  
class BlockChain(object):  
    ...  
  
from flask import Flask  
  
app = Flask(__name__)  
  
@app.route('/create-transaction', methods=['POST'])  
def create_transaction():  
    pass  
  
app.run(debug=True)

from argparse import ArgumentParser  
  
parser = ArgumentParser()  
parser.add_argument('-H', '--host', default='127.0.0.1')  
parser.add_argument('-p', '--port', default=5000, type=int)  
args = parser.parse_args()
app.run(host=args.host, port=args.port, debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


OSError: [Errno 98] Address already in use

In [30]:
from uuid import uuid4  
from flask import Flask  
  
app = Flask(__name__)  
  
blockchain = BlockChain()  
  
node_address = uuid4().hex  # Unique address for current node  
  
@app.route('/create-transaction', methods=['POST'])  
def create_transaction():  
    pass  
  
@app.route('/mine', methods=['GET'])  
def mine():  
    pass  
  
@app.route('/chain', methods=['GET'])  
def get_full_chain():  
    pass  
  
  
from argparse import ArgumentParser  
parser = ArgumentParser()  
parser.add_argument('-H', '--host', default='127.0.0.1')  
parser.add_argument('-p', '--port', default=5000, type=int)  
args = parser.parse_args()  
  
app.run(host=args.host, port=args.port, debug=True)

usage: ipykernel_launcher.py [-h] [-H HOST] [-p PORT]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/virus/.local/share/jupyter/runtime/kernel-d9b347ec-0c2b-46a0-8be6-d5a04d4abec5.json


SystemExit: 2

In [31]:
from flask import Flask, jsonify

curl http://127.0.0.1:5000/chain 

SyntaxError: invalid syntax (<ipython-input-31-39968ac41cb0>, line 3)

In [32]:
curl -X POST -H "Content-Type: application/json" -d '{"sender": "addr1", "recipient": "addr2", "amount": 3}' "http://127.0.0.1:5000/create-transaction"  



SyntaxError: invalid syntax (<ipython-input-32-53b2c6256975>, line 1)

In [33]:
@app.route('/register-node', methods=['POST'])  
def register_node():  
    node_data = request.get_json()  
    blockchain.create_node(node_data.get('address'))  
    response = {  
        'message': 'New node has been added',  
        'node_count': len(blockchain.nodes),  
        'nodes': list(blockchain.nodes),  
    }  
    return jsonify(response), 201

In [34]:
def get_neighbour_chains():  
    neighbour_chains = []  
  
    for node_address in blockchain.nodes:  
        resp = requests.get(node_address + url_for('get_full_chain')).json()  
        chain = resp['chain']  
        neighbour_chains.append(chain)  
  
    return neighbour_chains

In [35]:
node_address + url_for('get_full_chain') # equals to 'http://127.0.0.1:5000 + '/chain'


NameError: name 'url_for' is not defined

In [36]:
@app.route('/sync-chain', methods=['GET'])
def consensus():
    neighbour_chains = get_neighbour_chains()
    if not neighbour_chains:
       return jsonify({'message': 'No neighbour chain is available'})
    
    longest_chain = max(neighbour_chains, key=len) # Get the longest chain

    if len(blockchain.chain) >= len(longest_chain):  # If our chain is longest, then do nothing
        response = {
            'message': 'Chain is already up to date',
            'chain': blockchain.get_serialized_chain
        }
    else:  # If our chain isn't longest, then we store the longest chain
        blockchain.chain = [blockchain.get_block_object_from_block_data(block) for block in longest_chain]
        response = {
           'message': 'Chain was replaced',
           'chain': blockchain.get_serialized_chain
        }

    return jsonify(response)

In [37]:
def register_node(node_addr, parent_server):  
    requests.post(parent_server + '/register-node', json={'address': node_addr})  
    print("\\nOn Server {}: Node-{} has been registered successfully!\\n".format(parent_server, node_addr))  
  
def create_transaction(server, data):  
    requests.post(server + '/create-transaction', json=data).json()  
    print("On Server {}: Transaction has been processed!\\n".format(server))  
  
def mine_block(server):  
    requests.get(server + '/mine').json()  
    print("On Server {}: Block has been mined successfully!\\n".format(server))  
  
def get_server_chain(server):  
    resp = requests.get(server + '/chain').json()  
    print("On Server {}: Chain is-\\n{}\\n".format(server, resp))  
    return resp  
  
def sync_chain(server):  
    print("On Server {}: Started Syncing Chain . . .".format(server))  
    resp = requests.get(server + '/sync-chain')  
    print("On Server {}: Chain synced!\\n".format(server))  
  
  
# two servers are running on 5000 and 5001 port  
  
server1 = 'http://127.0.0.1:5000'  
server2 = 'http://127.0.0.1:5001'  
  
register_node(server2, server1) # server2 node will be register inside server1  
  
create_transaction(server2, {'sender': 'I', 'recipient': 'you', 'amount': 3})  
  
mine_block(server2) # Mined a new block on server2  
  
get_server_chain(server1) # server1's chain  
get_server_chain(server2) # server2's chain  
  
sync_chain(server1) # updating server1's chain with neighbour node's chain  
  
get_server_chain(server1) # server1's chain after syncing

NameError: name 'requests' is not defined